# Model training and registration
This notebook show the process for training the model, converting the model to ONNX and uploading the ONNX model to Azure Storage.

## Explore the training data
The following cells load the source CSV file into a Spark DataFrame and create a temporary view that can be used to query the data with Spark SQL.


In [35]:
df = spark.read.load('abfss://dev@<primary_storage>.dfs.core.windows.net/bronze/wwi-factsale.csv', format="csv"
## If header exists uncomment line bellow
, header=True, sep="|"
)

In [36]:
df.createOrReplaceTempView("facts")

In [37]:
display(spark.sql("SELECT * FROM facts WHERE `Customer Key` == '11' ORDER BY `Stock Item Key`"))

## Predict Quantity given Customer Key and Stock Item Key
In the following cells we load a subset of the data that just contains the fields needed for training. 



In [38]:
from pyspark.sql.functions import col
df3 = spark.sql("SELECT double(`Customer Key`) as customerkey, double(`Stock Item Key`) as stockitemkey, double(`Quantity`) as quantity FROM facts").where(col("quantity").isNotNull())
df3.cache()
print("Number of records:", df3.count())
df3.show(10)

Now, we split our DataFrame into training and testing DataFrames.


In [39]:
trainingFraction = 0.7
testingFraction = (1-trainingFraction)
seed = 42

# Split the dataframe into test and training dataframes
df_train, df_test = df3.randomSplit([trainingFraction, testingFraction], seed=seed)

Next, we package the data into the format expected by SPark ML's LinearRegression. It requires a DataFrame with two columns- `features` and a column with the labels to predict (`quantity` in this case).


In [40]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols = ['customerkey', 'stockitemkey'], outputCol = 'features')

In the following cell, we create the LinearRegression model.

In [41]:
from pyspark.ml.regression import LinearRegression

lin_reg = LinearRegression(featuresCol = 'features', labelCol='quantity', maxIter = 10, regParam=0.3)

Next, we build the training pipeline.

In [42]:
from pyspark.ml import Pipeline

stages = []
stages += [vectorAssembler]
stages += [lin_reg]

partialPipeline = Pipeline().setStages(stages)

In the following cell, we train our LinearRegression model.


In [43]:
pipelineModel = partialPipeline.fit(df_train)

print("Coefficients: " + str(pipelineModel.stages[-1].coefficients))
print("Intercept: " + str(pipelineModel.stages[-1].intercept))

With a trained model in hand, we can use it to make predictions against the test DataFrame.


In [44]:
df_pred = pipelineModel.transform(df_test)

from pyspark.sql.functions import rand
display(df_pred.select(["quantity",  "prediction"]).orderBy(rand()).limit(5))

## Convert model to ONNX
In the cells that follow, we convert the model to ONNX and show how an output of how ONNX represents the Spark ML model.


In [53]:
from onnxmltools import convert_sparkml
from onnxmltools.convert.common.data_types import FloatTensorType

initial_types = [ 
    ("customerkey", FloatTensorType([1, 1])),
    ("stockitemkey", FloatTensorType([1, 1]))
]

In [54]:
model_onnx = convert_sparkml(pipelineModel, 'sparkml GeneralizedLinearRegression', initial_types)
model_onnx

## Upload the model to Azure Storage
In the cells that follow we save the ONNX model to the storage of Spark driver node temporarily. Then we use the Azure Storage Python SDK to upload the ONNX model to Azure Storage.


In [47]:
with open("model.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())

In [48]:
connection_string = "DefaultEndpointsProtocol=https;AccountName=<blob_storage>;AccountKey=<blob_storage_account_key>;EndpointSuffix=core.windows.net"

from azure.storage.blob import BlobClient

blob = BlobClient.from_connection_string(conn_str=connection_string, container_name="models", blob_name="onnx/model.onnx")

with open("./model.onnx", "rb") as data:
    blob.upload_blob(data,overwrite=True)